[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/paraview_hello_world.ipynb)

### TODO #1417 add description
### TODO #1490 fix installation on binder
For more informations about Paraview please refere to [Paraview documentation](https://docs.paraview.org/en/latest/Tutorials/SelfDirectedTutorial/)

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
import os
import subprocess
SUBPROCESS_ENV = os.environ.copy()

if 'google.colab' in sys.modules:
    !apt-get install -qq ghostscript
    !wget -nv "https://paraview.org/paraview-downloads/download.php?submit=Download&version=v5.13&type=binary&os=Linux&downloadFile=ParaView-5.13.1-egl-MPI-Linux-Python3.10-x86_64.tar.gz" -O paraview.tar.gz
    !tar xzf paraview.tar.gz
    SUBPROCESS_ENV['PATH'] += ':' + subprocess.check_output(['bash', '-c', "echo `pwd`/`dirname ParaView*/bin/pvpython`"], text=True)[:-1]
    
    # check if Ghostscript's ps2pdf works
    assert subprocess.check_call(['type', 'ps2pdf'], shell=True) == 0
    
# check if Paraview's pvpython works
assert subprocess.check_call(['pvpython', '--version'], env=SUBPROCESS_ENV) == 0
assert subprocess.check_call(['pvpython', '-c', 'import paraview'], env=SUBPROCESS_ENV) == 0

paraview version 5.13.1


In [3]:
from PySDM_examples.Arabas_et_al_2015 import Settings, SpinUp
from PySDM_examples.Szumowski_et_al_1998 import Simulation, Storage
from PySDM.exporters import VTKExporter
from PySDM_examples.utils import ProgBarController
from PySDM import products as PySDM_products
import glob
import platform

In [4]:
products = [
    PySDM_products.EffectiveRadius(unit='um')
]

##### 1. run a simulations saving output to VTK files

In [5]:
settings = Settings()
storage = Storage()
simulation = Simulation(settings, storage, SpinUp=SpinUp)
simulation.reinit(products)

vtk_exporter = VTKExporter(path='.')    

simulation.run(ProgBarController("progress:"), vtk_exporter=vtk_exporter)
vtk_exporter.write_pvd()

FloatProgress(value=0.0, description='progress:', max=1.0)

#### 2. create a Python script for Paraview

In [6]:
%%writefile pvscript.py

from paraview import simple as pvs

# load data
reader_prod = pvs.OpenDataFile("./output/sd_products.pvd")
reader_attr = pvs.OpenDataFile("./output/sd_attributes.pvd")

# prepare view settings
view = pvs.GetRenderView()
view.ViewSize = [2000, 800]
view.Background = [1, 1, 1]
view.CenterAxesVisibility = False
view.OrientationAxesVisibility = False
axesGrid = view.AxesGrid
axesGrid.Visibility = True
axesGrid.XTitle = 'Z [m]'
axesGrid.YTitle = 'X [m]'

axesGrid.XAxisUseCustomLabels = True
axesGrid.XAxisLabels = [300, 600, 900, 1200]
axesGrid.YAxisUseCustomLabels = True
axesGrid.YAxisLabels = [300, 600, 900, 1200]

axesGrid.XTitleFontSize = 30
axesGrid.XLabelFontSize = 30
axesGrid.YTitleFontSize = 30
axesGrid.YLabelFontSize = 30

axesGrid.XTitleColor = [0, 0, 0]
axesGrid.XLabelColor = [0, 0, 0]
axesGrid.YTitleColor = [0, 0, 0]
axesGrid.YLabelColor = [0, 0, 0]
axesGrid.GridColor = [0.1, 0.1, 0.1]

# render particles
var = 'radius'
multiplier = 1e6
palette = 'Cold and Hot'
palette_invert = False
color_range = [0, 10]
logscale = False
title = var + ' [um]'

calculator = pvs.Calculator(reader_attr)
calculator.Function = f'{var}*{multiplier}'
display_attr = pvs.Show(calculator, view)

display_attr.SetRepresentationType('Point Gaussian')
display_attr.ShaderPreset = 'Sphere'
display_attr.GaussianRadius = 5
display_attr.MapScalars = 1

display_attr.Ambient = .25
pvs.ColorBy(display_attr, ('POINTS', 'Result'))
color_scale_attr = pvs.GetColorTransferFunction('Result')
color_scale_attr.ApplyPreset(palette, True)
if palette_invert:
    color_scale_attr.InvertTransferFunction()
if color_range is None:
    display_attr.RescaleTransferFunctionToDataRange(True)
else:
    color_scale_attr.RescaleTransferFunction(color_range)
if logscale:
    color_scale_attr.MapControlPointsToLogSpace()
    color_scale_attr.UseLogScale = 1
colorbar_attr = pvs.GetScalarBar(color_scale_attr, view)
colorbar_attr.TitleColor = [0, 0, 0]
colorbar_attr.LabelColor = [0, 0, 0]
colorbar_attr.Title = title
colorbar_attr.ComponentTitle = ''
colorbar_attr.TitleFontSize = 30
colorbar_attr.LabelFontSize = 30
colorbar_attr.Visibility = True
colorbar_attr.WindowLocation = 'Any Location'
colorbar_attr.Position = [.1, .333]
colorbar_attr.RangeLabelFormat = '%g'

# render product
var = 'effective radius'
palette = 'X Ray'
palette_invert = True
color_range = [0, 10]
logscale = False
title = var + ' [um]'

display_prod = pvs.Show(reader_prod)
display_prod.SetRepresentationType('Surface')
display_prod.Ambient = .25
pvs.ColorBy(display_prod, ('CELLS', var))
color_scale_prod = pvs.GetColorTransferFunction(var)
if color_range is None:
    display_prod.RescaleTransferFunctionToDataRange(True)
else:
    color_scale_prod.RescaleTransferFunction(color_range)
color_scale_prod.ApplyPreset(palette, True)
if palette_invert:
    color_scale_prod.InvertTransferFunction()
colorbar_prod = pvs.GetScalarBar(color_scale_prod, view)
colorbar_prod.TitleColor = [0, 0, 0]
colorbar_prod.LabelColor = [0, 0, 0]
colorbar_prod.Title = title
colorbar_prod.ComponentTitle = ''
colorbar_prod.TitleFontSize = 30
colorbar_prod.LabelFontSize = 30
colorbar_prod.Visibility = True
colorbar_prod.Position = [.92, .333]
colorbar_prod.WindowLocation = 'Any Location'
colorbar_prod.RangeLabelFormat = '%g'

# compose the scene
scene = pvs.GetAnimationScene()
scene.UpdateAnimationUsingDataTimeSteps()
pvs.Render(view)
cam = pvs.GetActiveCamera()
cam.SetViewUp(1, 0, 0)
pos = list(cam.GetPosition())
pos[-1] = -pos[-1]
cam.SetPosition(pos)
cam.Dolly(1.45)

# save animation to an Ogg Vorbis file
pvs.SaveAnimation('output/anim.ogv', view, FrameRate=10)

# save animation frame as pdfs
for t in reader_prod.TimestepValues:
    view.ViewTime = t
    for reader in (reader_prod, reader_attr):
        reader.UpdatePipeline(t)    
    pvs.ExportView(
        filename=f'output/anim_frame_{t}.pdf',
        view=view,
        Rasterize3Dgeometry= False,
        GL2PSdepthsortmethod= 'BSP sorting (slow, best)',
    )

Overwriting pvscript.py


#### 3. execute the script using `pvpython`

In [7]:
subprocess.run(['pvpython', '--force-offscreen-rendering', 'pvscript.py'], check=platform.system() != 'Windows', env=SUBPROCESS_ENV)

CompletedProcess(args=['pvpython', '--force-offscreen-rendering', 'pvscript.py'], returncode=0)

#### 4. reduce file size for generated pdf files

In [8]:
if platform.system() != 'Windows':
    for file in glob.glob('output/anim_frame_*.pdf'):
        subprocess.run(['ps2pdf', file, file+'_'], capture_output=True, check=True)
        subprocess.run(['mv', file+'_', file], check=True)